In [ ]:
import sys

sys.path

sys.executable

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from sklearn import preprocessing
import random
import tensorflow as tf
from tensorflow import keras

np.random.seed(1337)


from keras.preprocessing import sequence
from tensorflow.keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.convolutional import Conv1D


%matplotlib inline

### Parameters for plotting model results ###
pd.set_option("display.max_colwidth",100)
sns.set(style="ticks", color_codes=True)
plt.rcParams['font.weight'] = 'normal'
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['axes.labelpad'] = 5
plt.rcParams['axes.linewidth']= 2
plt.rcParams['xtick.labelsize']= 14
plt.rcParams['ytick.labelsize']= 14
plt.rcParams['xtick.major.size'] = 6
plt.rcParams['ytick.major.size'] = 6
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.minor.width'] = 1
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['axes.edgecolor'] = 'black'


def train_model(x, y, border_mode='same', inp_len=133, nodes=40, layers=3, filter_len=8, nbr_filters=120,
                dropout1=0, dropout2=0, dropout3=0, nb_epoch=3):
    ''' Build model archicture and fit.'''
    model = Sequential()
    if layers >= 1:
        model.add(Conv1D(activation="relu", input_shape=(inp_len, 4), padding=border_mode, filters=nbr_filters, kernel_size=filter_len))
    if layers >= 2:
        model.add(Conv1D(activation="relu", input_shape=(inp_len, 1), padding=border_mode, filters=nbr_filters, kernel_size=filter_len))
        model.add(Dropout(dropout1))
    if layers >= 3:
        model.add(Conv1D(activation="relu", input_shape=(inp_len, 1), padding=border_mode, filters=nbr_filters, kernel_size=filter_len))
        model.add(Dropout(dropout2))
    model.add(Flatten())

    model.add(Dense(nodes))
    model.add(Activation('relu'))
    model.add(Dropout(dropout3))
    
    model.add(Dense(1))
    model.add(Activation('linear'))

    #compile the model
    adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss='mean_squared_error', optimizer=adam)
    

    model.fit(x, y, batch_size=128, epochs=nb_epoch, verbose=1)
    

    return model


def test_data(df, model, test_seq, obs_col, output_col='pred'):
    '''Predict mean ribosome load using model and test set UTRs'''
    
    # Scale the test set mean ribosome load
    scaler = preprocessing.StandardScaler() #sets mrl based on variance rather than mean
    scaler.fit(df[obs_col].values.reshape(-1,1))
    
    # Make predictions
    predictions = model.predict(test_seq).reshape(-1,1)
    
    # Inverse scaled predicted mean ribosome load and return in a column labeled 'pred'
    df.loc[:,output_col] = scaler.inverse_transform(predictions)
    return df


def one_hot_encode(df, col='Alligned Barcode', seq_len=133):
    # Dictionary returning one-hot encoding of nucleotides. 
    nuc_d = {'a':[1,0,0,0],'c':[0,1,0,0],'g':[0,0,1,0],'t':[0,0,0,1], 'n':[0,0,0,0]}
    
    # Creat empty matrix.
    vectors=np.empty([len(df),seq_len,4])
    
    # Iterate through UTRs and one-hot encode
    for i,seq in enumerate(df[col].str[:seq_len]): 
        seq = seq.lower()
        a = np.array([nuc_d[x] for x in seq])
        vectors[i] = a
    return vectors


def r2(x,y):
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    return r_value**2

print("success")

In [14]:
df = pd.read_pickle('More Barcodes From Fitz/Alligned Barcodes-50bp flanks/mg_all_barcodes_133bp_Alligned.pkl')       #mod1-added my own file
df.sort_values('Frequency', inplace=True, ascending=False)
df.reset_index(inplace=True, drop=True)
df = df.iloc[:10098] 


e_test = df.iloc[:1010]  
e_train = df.iloc[1010:]  

# One-hot encode both training and test UTRs
seq_e_train = one_hot_encode(e_train,seq_len=133)
seq_e_test = one_hot_encode(e_test, seq_len=133)

# Scale the training mean ribosome load values
e_train.loc[:, 'Frequency']

1010     0.000448
1011     0.000448
1012     0.000448
1013     0.000448
1014     0.000448
           ...   
10093    0.000052
10094    0.000052
10095    0.000052
10096    0.000052
10097    0.000052
Name: Frequency, Length: 9088, dtype: float64

In [15]:
model = train_model(seq_e_train, e_train['Frequency'], nb_epoch=3,border_mode='same',
                   inp_len=133, nodes=40, layers=3, nbr_filters=120, filter_len=8, dropout1=0,
                   dropout2=0,dropout3=0.2)

Epoch 1/3
71/71 [==============================] - 20s 285ms/step - loss: 0.0206
Epoch 2/3
71/71 [==============================] - 20s 285ms/step - loss: 3.2037e-06
Epoch 3/3
71/71 [==============================] - 20s 285ms/step - loss: 2.3683e-06


In [4]:
model.save('training_MRL_CNN_test.hdf5')

In [5]:
model = keras.models.load_model('training_MRL_CNN_test.hdf5')

In [16]:
e_test = test_data(df=e_test, model=model, obs_col='Frequency',test_seq=seq_e_test)
r = r2(e_test['Frequency'], e_test['pred'])
print ('r-squared = ', r)


e_test['pred']


r-squared =  0.0001839410016293923


/tmp/ipykernel_1893977/2638729819.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,output_col] = scaler.inverse_transform(predictions)


0       0.003062
1       0.003062
2       0.003061
3       0.003062
4       0.003061
          ...   
1005    0.003063
1006    0.003061
1007    0.003063
1008    0.003059
1009    0.003060
Name: pred, Length: 1010, dtype: float32